# Differential operators as matrices

Differential operators are linear operators on vector fields of functions. They have many similar properties to matrices on finite-dimensional vector spaces. Indeed, if a function $y(x)$ is sampled at discrete points with values $y_n$, then the approximations to $y'$, $y''$ are created by multiplying the vector $(y_n)$ with a difference matrix.

In particular, the eigenvalues and eigenfunctions of an operator can be approximated by looking at the eigenvalues and eigenvectors of the discretised matrix operator. This can be done numerically using methods such as `eig` from the `scipy` package: see [its documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eig.html) for information about its algorithm. (We are only interested in self-adjoint operators. When discretised, they become Hermitian matrices.) Note that most such algorithms return the eigenvectors as normalised vectors; their actual magnitude when interpreted as functions needs to be scaled by $\sqrt{N}$, where $N$ is the number of gridpoints.

In [ ]:
import numpy as np
from numpy import pi
from numpy import sqrt, sin, cos, tan, sinh, cosh, exp, dot
# from numpy.linalg import eig, eigh
from scipy.linalg import eig, eigh
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def discretise(xa, xb, nx=101, bcs="periodic"):
    """Returns the xs, the spacing dx between the xs,
    and the identity and first and second derivative
    matrices, computed using a 2nd-order scheme."""
    xs = np.linspace(xa, xb, nx)
    dx = xs[1] - xs[0]

    eye = np.eye(nx)
    
    # Use central finite differences
    ddx = (np.roll(eye, 1, 1) - np.roll(eye, -1, 1)) / (2*dx)
    d2dx2 = (np.roll(eye, 1, 1) - 2*eye + np.roll(eye, -1, 1)) / (dx*dx)
    
    if bcs == "periodic":
        pass
    elif bcs == "dirichlet":
        # Use forward or backward schemes
        # Would it help to use 3rd-order scheme at boundaries?
        ddx[0, 4:] = 0; ddx[0, :4] = (-11/6, 3, -3/2, 1/3) / dx
        ddx[-1, :-4] = 0; ddx[-1, -4:] = (-1/3, 3/2, -3, 11/6) / dx
        
        d2dx2[0, 4:] = 0; d2dx2[0, :4] = (2, -5, 4, -1) / (dx*dx)
        d2dx2[-1, :-4] = 0; d2dx2[-1, -4:] = (-1, 4, -5, 2) / (dx*dx)
    
    elif bcs == "neumann":
        # Use forward or backward schemes
        eye[0, 0] = dx*dx; eye[-1, -1] = dx*dx
        ddx[0, 4:] = 0; ddx[0, :4] = (-11/6, 3, -3/2, 1/3) / dx
        ddx[-1, :-4] = 0; ddx[-1, -4:] = (-1/3, 3/2, -3, 11/6) / dx
        
        d2dx2[0, 4:] = 0; d2dx2[0, :4] = (2, -5, 4, -1) / (dx*dx)
        d2dx2[-1, :-4] = 0; d2dx2[-1, -4:] = (-1, 4, -5, 2) / (dx*dx)
    
    else:
        raise ValueError("Unsupported BCs")
    
    return xs, dx, eye, ddx, d2dx2

def myeig(op, eye, *args, **kwargs):
    nx = op.shape[0]
    eigvals, eigvecs = eig(op, eye, *args, **kwargs)
    ordering = sorted(range(nx), key=lambda i: np.real(eigvals)[i])
    eigvals = eigvals[ordering]
    eigvecs = eigvecs[:, ordering] * sqrt(nx)
    return eigvals, eigvecs

## The harmonic functions
These are the eigenfunctions of the operator $-D^2$, subject to either Dirichlet or Neumann boundary conditions.

In [ ]:
xa = 0; xb = pi; nx = 101;
xs, dx, eye, ddx, d2dx2 = discretise(xa, xb, nx)

op = -d2dx2; 
op[0, :] = 0;
op[-1, :] = 0;
# op[0, 0:3] = (-3/2, 2, -1/2)/(dx*dx)
# # op[0, 0:2] = (-1, 1)/(dx*dx*dx)
# op[-1, -1] = 1/(dx*dx)
eye[0, 3:] = 0; eye[0, :3] = (3/2, -2, 1/2) / dx;
eigvals, eigvecs = myeig(op, eye)

@interact(mode=widgets.IntSlider(min=0, max=20, continuous_update=True))
def harmfun(mode):
    plt.plot(xs, np.real(eigvecs[:, mode]) * np.sign(np.real(eigvecs[0, mode])))
    plt.gca().set(title=f"E = %.2f" % eigvals[mode], 
                  xlim=[xa, xb], ylim=[-2, 2])
    plt.gca().grid()

## The quantum harmonic oscillator

In suitable units, the time-independent Schrodinger equation for the wavefunction in a harmonic potential is 
$$ -y'' + x^2 y = E y.$$
The possible energy levels $E$ are the eigenvalues of the operator $-D^2 + x^2$. The operator 'multiply by $x^2$' is represented as a diagonal matrix whose elements are $x_n^2$, where $x_n$ are the gridpoints of the discretisation.

In [ ]:
xa = -10; xb = 10; nx = 501;
xs, dx, eye, ddx, d2dx2 = discretise(xa, xb, nx)
d2dx2[0, :] = 0; d2dx2[-1, :] = 0 # impose boundary conditions
eigvals, eigvecs = eigh(-d2dx2 + np.diag(xs**2))
# eigvals, eigvecs = eigh(-d2dx2)

@interact(mode=widgets.IntSlider(min=0, max=12))
def qhoplot(mode):
    # TODO what's up with the first mode? 
    # I think we aren't imposing BCs correctly.
    plt.plot(xs, eigvecs[:, mode+1])
    plt.gca().set(title=f"E = %.2f" % eigvals[mode+1], 
                  xlim=[-10, 10], ylim=[-.16, .16])
    plt.gca().grid()
#     plt.plot(eigvals[:mode], 'x')

## Airy functions

In [ ]:
xa = -20; xb = 20; nx = 501;
xs, dx, eye, ddx, d2dx2 = discretise(xa, xb, nx)
# d2dx2[0, :] = 0; d2dx2[-1, :] = 0 # impose boundary conditions
eigvals, eigvecs = eigh(-d2dx2 + np.diag(xs))
# eigvals, eigvecs = eigh(-d2dx2)

@interact(mode=widgets.IntSlider(min=6, max=37, continuous_update=False))
def airy(mode):
    # TODO what's up with the first mode? 
    # I think we aren't imposing BCs correctly.
    plt.plot(xs, eigvecs[:, mode+1])
    plt.gca().set(title=f"E = %.2f" % eigvals[mode+1], 
                  xlim=[-10, 10], ylim=[-.16, .16])
    plt.gca().grid()
#     plt.plot(eigvals[:mode], 'x')

## Sheet 1, question 7

$$ y'' + 4y' + 4y = -\lambda y $$

so the operator of interest is $D^2 + 4D + 4I$. Note that this is _not_ a self-adjoint operator, so we'll need to use the `eig` solver, not `eigh`.

In [ ]:
xa = 0; xb = 1; nx = 301;
xs, dx, eye, ddx, d2dx2 = discretise(xa, xb, nx)
d2dx2[0, :] = 0; d2dx2[-1, :] = 0

In [ ]:
eigvals, eigvecs = eig(d2dx2 + 4*ddx + 4*eye)
eigvals = -eigvals
# eigvals, eigvecs = eigh(-d2dx2)

@interact(mode=widgets.IntSlider(min=0, max=12))
def s1q7_plot_notsl(mode):
    # TODO what's up with the first mode? 
    # I think we aren't imposing BCs correctly.
    plt.plot(xs[0::2], 
             eigvecs[0::2, mode+1] * sqrt(nx) * np.sign(eigvecs[2, mode+1]))
    plt.gca().set(title=f"E = %.2f" % eigvals[-mode-2], 
                  xlim=[0, 1], ylim=[-3, 3])
    plt.gca().grid()
#     plt.plot(eigvals[:mode], 'x')

We can turn the equation into a self-adjoint equation by multiplying through by the integrating factor $p(x) = \exp(4x)$, to get

$$ (py')' + 4py = -\lambda py. $$

As a matrix equation, this becomes

$$ 
(DPD + 4P)y = -\lambda Py,
$$
a generalised eigenvector problem. As in the QHO example, $P$ is a diagonal matrix whose elements are $p(x_n) = \exp(4x_n)$, representing multiplication by $p(x)$. So, recasting in Sturm-Liouville form is akin to diagonalising a matrix.

In [ ]:
p_mat = np.diag(exp(4*xs))
eigvals, eigvecs = eigh(ddx @ p_mat @ ddx + 4 * p_mat, p_mat)
eigvals = -eigvals

In [ ]:
@interact(mode=widgets.IntSlider(min=0, max=12))
def s1q7_plot_sl(mode):
    # TODO what's up with the first mode? 
    # I think we aren't imposing BCs correctly.
    plt.plot(xs[0::4], eigvecs[0::4, 2*mode+2] * sqrt(nx) * np.sign(eigvecs[4, 2*mode+2]))
    plt.gca().set(title=f"E = %.2f" % eigvals[-2*mode-4], 
                  xlim=[0, 1], ylim=[-1, 1])
    plt.gca().grid()
#     plt.plot(eigvals[:mode], 'x')

## Sheet 1, question 8: Bessel functions

In [ ]:
xa = 0; xb = 10; nx = 101;
xs, dx, eye, ddx, d2dx2 = discretise(xa, xb, nx)

x_mat = np.diag(xs)
bessel_operator = -dot(x_mat, d2dx2) - ddx
bessel_operator[0, :] = 0
bessel_operator[-1, :] = 0
eye = x_mat;
eye[0, :2] = (-1, 1)
eye[-1, -1] = 1

eigvals, eigvecs = myeig(bessel_operator, eye)
# eigvals = -eigvals
# eigvals, eigvecs = eigh(-d2dx2)

@interact(mode=widgets.IntSlider(min=0, max=12))
def s1q8_plot_notsl(mode):
    # TODO what's up with the first mode? 
    # I think we aren't imposing BCs correctly.
    plt.plot(xs, eigvecs[:, mode+2] / eigvecs[0, mode+2])
    plt.gca().set(title=f"E = %.2f" % eigvals[mode+2], 
                  xlim=[xa, xb], ylim=[-1.1, 1.1])
    plt.gca().grid()
#     plt.plot(eigvals[:mode], 'x')

## Sheet 1, question 9: Higher-order self-adjoint form

The differential operator of interest here is simply $D^4$. 

In [ ]:
def d4dx4_mat(nx, dx=1, bcs="dirichlet"):
    eye = np.eye(nx)
    d4dx4 = (np.roll(eye, 2, 1) - 4*np.roll(eye, 1, 1) + 6*eye 
                 - 4*np.roll(eye, -1, 1) + np.roll(eye, -2, 1)) / dx**4
    
    if bcs == "dirichlet":
        d4dx4[0, :] = 0;
        d4dx4[0, :5] = (1, -4, 6, -4, 1) / dx**4
        d4dx4[1, :] = 0;
        d4dx4[-2, :] = 0;
        d4dx4[-1, :] = 0;
    elif bcs == "periodic":
        pass
    else:
        raise ValueError
        
    return d4dx4

In [ ]:
xa = 0.0; xb = 1; nx = 257;
xs, dx, eye, ddx, d2dx2 = discretise(xa, xb, nx)
d2dx2[0, :] = 0; d2dx2[-1, :] = 0
eigvals, eigvecs = eigh(d4dx4_mat(nx, dx, "dirichlet"))

@interact(mode=widgets.IntSlider(min=1, max=12, continuous_update=False))
def s1q9_plot_sl(mode):
    fig, axs = plt.subplots(1, 2, figsize=[14, 4])
    axs[0].plot(range(1,18), eigvals[3:20] ** (1/4), 'kx')
    axs[0].set_xlabel('n')
    axs[0].set_ylabel('lambda_n ^ (1/4)')
    axs[0].grid()
    axs[0].set_title('lambda_n = O(n^4)')
    
    axs[1].plot(xs, eigvecs[:, mode+2] * sqrt(nx) * np.sign(eigvecs[128, mode+2]))
    axs[1].set(title=f"lambda = %.2f" % eigvals[mode+2], 
                  xlim=[0, 1], ylim=[-6, 6])
    axs[1].grid()
#     plt.plot(eigvals[:mode], 'x')

In [ ]:
plt.plot(np.real(eigvals))